In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [5]:
# Khai báo đường dẫn đến dataset đã được tăng cường trước đó
data_dir = r"C:\Users\nguye\Downloads\ML-DL\Dataset\AlzheimerData_Augmented"
# Định nghĩa các phép biến đổi dữ liệu
# Đây là các phép biến đổi được áp dụng trong quá trình huấn luyện và đánh giá
# Chúng giúp chuẩn hóa và tiền xử lý ảnh để phù hợp với mô hình
data_transforms = {
    'train': transforms.Compose([
        # Nếu muốn tăng cường thêm trong quá trình huấn luyện, bạn có thể thêm vào đây
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
# Tải dữ liệu bằng ImageFolder
full_dataset = datasets.ImageFolder(data_dir, data_transforms['val_test']) 
class_names = full_dataset.classes
print(f"Các lớp tìm thấy: {class_names}")

Các lớp tìm thấy: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']


In [9]:
# Bước 4: Chia dataset thành các tập train, validation và test
dataset_size = len(full_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Áp dụng transform huấn luyện cho tập train
train_dataset.dataset.transform = data_transforms['train']

print(f"Tổng số mẫu: {dataset_size}")
print(f"Kích thước tập huấn luyện: {len(train_dataset)}")
print(f"Kích thước tập kiểm tra: {len(val_dataset)}")
print(f"Kích thước tập đánh giá: {len(test_dataset)}")

Tổng số mẫu: 268217
Kích thước tập huấn luyện: 187751
Kích thước tập kiểm tra: 40232
Kích thước tập đánh giá: 40234


In [10]:
# Tạo DataLoader để load dữ liệu theo lô (batch)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [11]:
# Tải mô hình ResNet-101
# Sử dụng pretrained=True để tải mô hình đã được huấn luyện trước trên ImageNet
model = models.resnet101(pretrained=True)

# Tùy chỉnh lớp cuối cùng của mô hình
# Số đặc trưng đầu vào (in_features) của lớp cuối cùng của ResNet-101
num_ftrs = model.fc.in_features
# Thay thế lớp cuối cùng bằng một lớp tuyến tính mới với 4 đầu ra (tương ứng với 4 lớp)
model.fc = nn.Linear(num_ftrs, len(class_names))

C:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\nguye/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|███████████████████████████████████████████████████████████████████████████████| 171M/171M [00:09<00:00, 18.9MB/s]


In [13]:
# Chuyển đổi mô hình sang GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [14]:
# Định nghĩa hàm mất mát và bộ tối ưu
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Bắt đầu quá trình huấn luyện
epochs = 10 # Số lượng vòng lặp huấn luyện

best_val_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    # Giai đoạn huấn luyện
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    history['train_loss'].append(epoch_loss)
    history['train_acc'].append(epoch_acc.item())
    print(f'Huấn luyện: Mất mát = {epoch_loss:.4f}, Độ chính xác = {epoch_acc:.4f}')

    # Giai đoạn kiểm tra (validation)
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(val_loader, desc="Validating"):
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(val_dataset)
    epoch_acc = running_corrects.double() / len(val_dataset)
    history['val_loss'].append(epoch_loss)
    history['val_acc'].append(epoch_acc.item())
    print(f'Kiểm tra: Mất mát = {epoch_loss:.4f}, Độ chính xác = {epoch_acc:.4f}')

    # Lưu mô hình tốt nhất
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print("Đã lưu mô hình tốt nhất!")

print("\n--- Huấn luyện hoàn tất ---")
print(f"Độ chính xác cao nhất trên tập kiểm tra: {best_val_acc:.4f}")

Epoch 1/10
----------


Training:   1%|▋                                                                   | 61/5868 [05:00<7:56:48,  4.93s/it]


KeyboardInterrupt: 

In [ ]:
# Đánh giá mô hình trên tập test
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
all_preds = []
all_labels = []

print("\n--- Đánh giá mô hình trên tập test ---")
for inputs, labels in tqdm(test_loader, desc="Testing"):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# In báo cáo phân loại và ma trận nhầm lẫn
print("\nBáo cáo phân loại:")
print(classification_report(all_labels, all_preds, target_names=class_names))

print("Ma trận nhầm lẫn:")
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)

In [ ]:
# Vẽ biểu đồ lịch sử huấn luyện
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='Độ chính xác huấn luyện')
plt.plot(history['val_acc'], label='Độ chính xác kiểm tra')
plt.title('Độ chính xác Huấn luyện và Kiểm tra')
plt.xlabel('Epoch')
plt.ylabel('Độ chính xác')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='Mất mát huấn luyện')
plt.plot(history['val_loss'], label='Mất mát kiểm tra')
plt.title('Mất mát Huấn luyện và Kiểm tra')
plt.xlabel('Epoch')
plt.ylabel('Mất mát')
plt.legend()

plt.show()